In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import mysql.connector
import numpy as np

In [2]:
! pip install mysql-connector-python
! pip install tabulate


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import mysql.connector
mydb = mysql.connector.connect(
 host="localhost",
 user="root",
 password="",


)


print(mydb)
mycursor = mydb.cursor(buffered=True)

In [4]:
import mysql.connector

connection = mysql.connector.connect(
  host = "gateway01.ap-southeast-1.prod.aws.tidbcloud.com",
  port = 4000,
  user = "27BWX9vSyueH94y.root",
  password = "q1GlXoLTU7A9BpHQ",
 
)

# **queries to get insights from 5 tables**

In [5]:
#overall female count
mycursor.execute("select count(Gender) as Female_Count from data_spark.customer_details where Gender='Female'")
out=mycursor.fetchall()
from tabulate import tabulate
print(tabulate(out,headers=[i[0] for i in mycursor.description],  tablefmt='psql'))

+----------------+
|   Female_Count |
|----------------|
|           7514 |
+----------------+


In [6]:
#overall male count
mycursor.execute("select count(Gender) as Male_Count from data_spark.customer_details where Gender='Male'")
out=mycursor.fetchall()
from tabulate import tabulate
print(tabulate(out,headers=[i[0] for i in mycursor.description],  tablefmt='psql'))

+--------------+
|   Male_Count |
|--------------|
|         7742 |
+--------------+


In [7]:
#count of customers in country wise
mycursor.execute("SELECT Country, COUNT(*) AS Customer_Count \
FROM data_spark.customer_details \
GROUP BY Country \
ORDER BY Customer_Count DESC;")
out = mycursor.fetchall()
from tabulate import tabulate
print(tabulate(out, headers=[i[0] for i in mycursor.description], tablefmt='psql'))

+----------------+------------------+
| Country        |   Customer_Count |
|----------------+------------------|
| United States  |             6828 |
| United Kingdom |             1944 |
| Canada         |             1553 |
| Germany        |             1473 |
| Australia      |             1420 |
| Netherlands    |              733 |
| France         |              670 |
| Italy          |              635 |
+----------------+------------------+


In [8]:
#overall count of customers
mycursor.execute("select count(distinct s.CustomerKey)  as customer_count from data_spark.sales_details s;")
out=mycursor.fetchall()
from tabulate import tabulate
print(tabulate(out,headers=[i[0] for i in mycursor.description],  tablefmt='psql'))

+------------------+
|   customer_count |
|------------------|
|            11887 |
+------------------+


In [9]:
#count of stores in country wise
mycursor.execute("""SELECT Country, COUNT(StoreKey) 
FROM data_spark.stores_details 
GROUP BY Country 
ORDER BY COUNT(StoreKey) DESC;""")
out = mycursor.fetchall()
from tabulate import tabulate
print(tabulate(out, headers=[i[0] for i in mycursor.description], tablefmt='psql'))

+----------------+-------------------+
| Country        |   COUNT(StoreKey) |
|----------------+-------------------|
| United States  |                24 |
| Germany        |                 9 |
| France         |                 7 |
| United Kingdom |                 7 |
| Australia      |                 6 |
| Canada         |                 5 |
| Netherlands    |                 5 |
| Italy          |                 3 |
| Online         |                 1 |
+----------------+-------------------+


In [10]:
#store wise sales
mycursor.execute(""" SELECT s.StoreKey, sd.Country, SUM(pd.Unit_Price_USD * s.Quantity) AS total_sales_amount
    FROM data_spark.product_details pd
    JOIN data_spark.sales_details s ON pd.ProductKey = s.ProductKey
    JOIN data_spark.stores_details sd ON s.StoreKey = sd.StoreKey
    GROUP BY s.StoreKey, sd.Country;
""")
out = mycursor.fetchall()
from tabulate import tabulate
print(tabulate(out, headers=[i[0] for i in mycursor.description], tablefmt='psql'))

+------------+----------------+----------------------+
|   StoreKey | Country        |   total_sales_amount |
|------------+----------------+----------------------|
|          0 | Online         |          1.14043e+07 |
|          1 | Australia      |     243030           |
|          2 | Australia      |      15176           |
|          4 | Australia      |     442475           |
|          5 | Australia      |     859678           |
|          6 | Australia      |     538782           |
|          8 | Canada         |          1.23223e+06 |
|          9 | Canada         |          1.33615e+06 |
|         10 | Canada         |          1.04319e+06 |
|         12 | France         |     183091           |
|         13 | France         |     150925           |
|         14 | France         |     105714           |
|         15 | France         |     205120           |
|         16 | France         |     199010           |
|         17 | France         |     159607           |
|         

In [11]:
#overall selling amount
mycursor.execute("""
    SELECT SUM(pd.Unit_Price_USD * sd.Quantity) AS total_sales_amount
    FROM data_spark.product_details pd
    JOIN data_spark.sales_details sd ON pd.ProductKey = sd.ProductKey;
""")
out = mycursor.fetchall()
from tabulate import tabulate
print(tabulate(out, headers=[i[0] for i in mycursor.description], tablefmt='psql'))

+----------------------+
|   total_sales_amount |
|----------------------|
|          5.57555e+07 |
+----------------------+


In [12]:
#brand count
mycursor.execute("""
   select Brand ,count(Brand) as brand_count from data_spark.product_details group by  Brand;
""")
out = mycursor.fetchall()
from tabulate import tabulate
print(tabulate(out, headers=[i[0] for i in mycursor.description], tablefmt='psql'))

+----------------------+---------------+
| Brand                |   brand_count |
|----------------------+---------------|
| A. Datum             |           132 |
| Adventure Works      |           192 |
| Contoso              |           710 |
| Fabrikam             |           267 |
| Litware              |           264 |
| Northwind Traders    |            47 |
| Proseware            |           244 |
| Southridge Video     |           192 |
| Tailspin Toys        |           144 |
| The Phone Company    |           152 |
| Wide World Importers |           173 |
+----------------------+---------------+


In [13]:
#cp and sp diffenecnce and profit
mycursor.execute("""
  select Unit_price_USD,Unit_Cost_USD,round((Unit_price_USD-Unit_Cost_USD),2) as diff,
round(((Unit_price_USD-Unit_Cost_USD)/Unit_Cost_USD)*100,2) as profit_percent
from data_spark.product_details;
""")
out = mycursor.fetchall()
from tabulate import tabulate
print(tabulate(out, headers=[i[0] for i in mycursor.description], tablefmt='psql'))

+------------------+-----------------+---------+------------------+
|   Unit_price_USD |   Unit_Cost_USD |    diff |   profit_percent |
|------------------+-----------------+---------+------------------|
|            12.99 |            6.62 |    6.37 |            96.22 |
|            12.99 |            6.62 |    6.37 |            96.22 |
|            14.52 |            7.4  |    7.12 |            96.22 |
|            21.57 |           11    |   10.57 |            96.09 |
|            21.57 |           11    |   10.57 |            96.09 |
|            21.57 |           11    |   10.57 |            96.09 |
|            21.57 |           11    |   10.57 |            96.09 |
|            59.99 |           30.58 |   29.41 |            96.17 |
|            59.99 |           30.58 |   29.41 |            96.17 |
|            59.99 |           30.58 |   29.41 |            96.17 |
|            59.99 |           30.58 |   29.41 |            96.17 |
|            77.68 |           35.72 |   41.96 |

In [14]:
#brand wise selling amount
mycursor.execute("""
select Brand,round(sum(pd.Unit_price_USD*sd.Quantity),2) as sales_amount
from data_spark.product_details pd join data_spark.sales_details sd on pd.ProductKey=sd.ProductKey group by Brand;
""")
out = mycursor.fetchall()
from tabulate import tabulate
print(tabulate(out, headers=[i[0] for i in mycursor.description], tablefmt='psql'))

+----------------------+------------------+
| Brand                |     sales_amount |
|----------------------+------------------|
| A. Datum             |      1.48621e+06 |
| Adventure Works      |      1.18499e+07 |
| Contoso              |      1.07923e+07 |
| Fabrikam             |      6.80789e+06 |
| Litware              |      2.6595e+06  |
| Northwind Traders    |      1.12607e+06 |
| Proseware            |      3.21263e+06 |
| Southridge Video     |      2.5786e+06  |
| Tailspin Toys        | 682731           |
| The Phone Company    |      5.38682e+06 |
| Wide World Importers |      9.1728e+06  |
+----------------------+------------------+


In [15]:
#Subcategory wise selling amount
mycursor.execute("""
select Subcategory,count(Subcategory) from data_spark.product_details group by Subcategory;
""")
out = mycursor.fetchall()
from tabulate import tabulate
print(tabulate(out, headers=[i[0] for i in mycursor.description], tablefmt='psql'))

+----------------------------------+----------------------+
| Subcategory                      |   count(Subcategory) |
|----------------------------------+----------------------|
| Air Conditioners                 |                   62 |
| Bluetooth Headphones             |                   50 |
| Boxed Games                      |                   46 |
| Camcorders                       |                  103 |
| Cameras & Camcorders Accessories |                   69 |
| Car Video                        |                   46 |
| Cell phones Accessories          |                   30 |
| Coffee Machines                  |                   74 |
| Computers Accessories            |                  201 |
| Desktops                         |                   45 |
| Digital Cameras                  |                  100 |
| Digital SLR Cameras              |                  100 |
| Download Games                   |                  120 |
| Fans                             |    

In [16]:
mycursor.execute("""
select Subcategory ,round(sum(pd.Unit_price_USD*sd.Quantity),2) as TOTAL_SALES_AMOUNT
from data_spark.product_details pd join data_spark.sales_details sd on pd.ProductKey=sd.ProductKey
 group by Subcategory order by TOTAL_SALES_AMOUNT desc;
""")
out = mycursor.fetchall()
from tabulate import tabulate
print(tabulate(out, headers=[i[0] for i in mycursor.description], tablefmt='psql'))

+----------------------------------+----------------------+
| Subcategory                      |   TOTAL_SALES_AMOUNT |
|----------------------------------+----------------------|
| Desktops                         |          9.90636e+06 |
| Televisions                      |          4.30872e+06 |
| Projectors & Screens             |          3.76752e+06 |
| Water Heaters                    |          3.54782e+06 |
| Camcorders                       |          3.35799e+06 |
| Laptops                          |          3.16478e+06 |
| Movie DVD                        |          3.13101e+06 |
| Touch Screen Phones              |          3.08346e+06 |
| Smart phones & PDAs              |          2.80566e+06 |
| Refrigerators                    |          2.15266e+06 |
| Washers & Dryers                 |          2.11985e+06 |
| Digital SLR Cameras              |          1.89464e+06 |
| Monitors                         |          1.51812e+06 |
| Bluetooth Headphones             |    

In [17]:
#country overall wise sales
mycursor.execute("""
select s.Country,sum(pd.Unit_price_USD*sd.Quantity) as total_sales from data_spark.product_details pd
join data_spark.sales_details sd on pd.ProductKey=sd.ProductKey 
join data_spark.stores_details s on sd.StoreKey=s.StoreKey group by s.Country ;
""")
out = mycursor.fetchall()
from tabulate import tabulate
print(tabulate(out, headers=[i[0] for i in mycursor.description], tablefmt='psql'))

+----------------+---------------+
| Country        |   total_sales |
|----------------+---------------|
| Australia      |   2.09914e+06 |
| Canada         |   3.61156e+06 |
| France         |   1.22955e+06 |
| Germany        |   4.24628e+06 |
| Italy          |   2.05909e+06 |
| Netherlands    |   1.59134e+06 |
| Online         |   1.14043e+07 |
| United Kingdom |   5.74977e+06 |
| United States  |   2.37644e+07 |
+----------------+---------------+


In [18]:
mycursor.execute("""
select s.Country,count(DISTINCT s.StoreKey),sum(pd.Unit_price_USD*sd.Quantity) as total_sales from data_spark.product_details pd
join data_spark.sales_details sd on pd.ProductKey=sd.ProductKey 
join data_spark.stores_details s on sd.StoreKey=s.StoreKey group by s.Country ;
""")
out = mycursor.fetchall()
from tabulate import tabulate
print(tabulate(out, headers=[i[0] for i in mycursor.description], tablefmt='psql'))

+----------------+------------------------------+---------------+
| Country        |   count(DISTINCT s.StoreKey) |   total_sales |
|----------------+------------------------------+---------------|
| Australia      |                            5 |   2.09914e+06 |
| Canada         |                            3 |   3.61156e+06 |
| France         |                            7 |   1.22955e+06 |
| Germany        |                            8 |   4.24628e+06 |
| Italy          |                            3 |   2.05909e+06 |
| Netherlands    |                            4 |   1.59134e+06 |
| Online         |                            1 |   1.14043e+07 |
| United Kingdom |                            7 |   5.74977e+06 |
| United States  |                           20 |   2.37644e+07 |
+----------------+------------------------------+---------------+


In [19]:
#year wise brand sales
mycursor.execute("""
select year(Order_Date),pd.Brand,round(SUM(Unit_price_USD*sd.Quantity),2) as year_sales FROM data_spark.sales_details sd
join data_spark.product_details pd on sd.ProductKey=pd.ProductKey group by year(Order_Date),pd.Brand;
""")
out = mycursor.fetchall()
from tabulate import tabulate
print(tabulate(out, headers=[i[0] for i in mycursor.description], tablefmt='psql'))

+--------------------+----------------------+------------------+
|   year(Order_Date) | Brand                |       year_sales |
|--------------------+----------------------+------------------|
|               2016 | A. Datum             | 231663           |
|               2016 | Adventure Works      |      1.59781e+06 |
|               2016 | Contoso              |      1.59061e+06 |
|               2016 | Fabrikam             | 974804           |
|               2016 | Litware              | 525197           |
|               2016 | Northwind Traders    | 178613           |
|               2016 | Proseware            | 243534           |
|               2016 | Southridge Video     | 379099           |
|               2016 | Tailspin Toys        |  49498.1         |
|               2016 | The Phone Company    | 368537           |
|               2016 | Wide World Importers | 807433           |
|               2017 | A. Datum             | 154171           |
|               2017 | Ad

In [20]:
#overall sales with quatity
mycursor.execute("""
select Brand,sum(pd.Unit_Price_USD*sd.Quantity) as sp,sum(pd.Unit_Cost_USD*sd.Quantity) as cp,
(SUM(pd.Unit_Price_USD*sd.Quantity) - SUM(pd.Unit_Cost_USD*sd.Quantity)) / SUM(pd.Unit_Cost_USD*sd.Quantity) * 100 as profit 
from data_spark.product_details pd join data_spark.sales_details sd on sd.ProductKey=pd.ProductKey
group by Brand;
""")
out = mycursor.fetchall()
from tabulate import tabulate
print(tabulate(out, headers=[i[0] for i in mycursor.description], tablefmt='psql'))

+----------------------+------------------+------------------+----------+
| Brand                |               sp |               cp |   profit |
|----------------------+------------------+------------------+----------|
| A. Datum             |      1.48621e+06 | 602705           |  146.589 |
| Adventure Works      |      1.18499e+07 |      4.91259e+06 |  141.215 |
| Contoso              |      1.07923e+07 |      4.47112e+06 |  141.379 |
| Fabrikam             |      6.80789e+06 |      2.74642e+06 |  147.883 |
| Litware              |      2.6595e+06  |      1.10573e+06 |  140.519 |
| Northwind Traders    |      1.12607e+06 | 476656           |  136.244 |
| Proseware            |      3.21263e+06 |      1.27734e+06 |  151.509 |
| Southridge Video     |      2.5786e+06  |      1.05572e+06 |  144.25  |
| Tailspin Toys        | 682731           | 309682           |  120.462 |
| The Phone Company    |      5.38682e+06 |      2.32906e+06 |  131.288 |
| Wide World Importers |      9.1728e+

In [21]:
#month wise sales with quatity
mycursor.execute("""
select month(Order_Date),sum(pd.Unit_Price_USD*sd.Quantity) as sp_month from data_spark.sales_details
 sd join data_spark.product_details pd on sd.ProductKey=pd.ProductKey
group by month(Order_Date);
""")
out = mycursor.fetchall()
from tabulate import tabulate
print(tabulate(out, headers=[i[0] for i in mycursor.description], tablefmt='psql'))

+---------------------+------------------+
|   month(Order_Date) |         sp_month |
|---------------------+------------------|
|                   1 |      6.75998e+06 |
|                   2 |      7.84248e+06 |
|                   3 |      2.62552e+06 |
|                   4 | 607334           |
|                   5 |      4.75798e+06 |
|                   6 |      4.29304e+06 |
|                   7 |      3.85242e+06 |
|                   8 |      4.08517e+06 |
|                   9 |      4.36386e+06 |
|                  10 |      4.31503e+06 |
|                  11 |      4.75606e+06 |
|                  12 |      7.49661e+06 |
+---------------------+------------------+


In [22]:
#month and year wise sales with quatity
mycursor.execute("""
select month(Order_Date),year(Order_Date),Brand,sum(pd.Unit_Price_USD*sd.Quantity) as sp_month from data_spark.sales_details
sd join data_spark.product_details pd on sd.ProductKey=pd.ProductKey
group by month(Order_Date),year(Order_Date),Brand;
""")
out = mycursor.fetchall()
from tabulate import tabulate
print(tabulate(out, headers=[i[0] for i in mycursor.description], tablefmt='psql'))

+---------------------+--------------------+----------------------+------------+
|   month(Order_Date) |   year(Order_Date) | Brand                |   sp_month |
|---------------------+--------------------+----------------------+------------|
|                   1 |               2016 | A. Datum             |   23110.5  |
|                   1 |               2016 | Adventure Works      |  122026    |
|                   1 |               2016 | Contoso              |  127559    |
|                   1 |               2016 | Fabrikam             |  127689    |
|                   1 |               2016 | Litware              |   65521.5  |
|                   1 |               2016 | Northwind Traders    |   26240.4  |
|                   1 |               2016 | Proseware            |   32056.1  |
|                   1 |               2016 | Southridge Video     |   51891.6  |
|                   1 |               2016 | Tailspin Toys        |    6279.02 |
|                   1 |     

In [23]:
#year wise sales
mycursor.execute("""
select year(Order_Date),sum(pd.Unit_Price_USD*sd.Quantity) as sp_month from data_spark.sales_details
 sd join data_spark.product_details pd on sd.ProductKey=pd.ProductKey
group by year(Order_Date);
""")
out = mycursor.fetchall()
from tabulate import tabulate
print(tabulate(out, headers=[i[0] for i in mycursor.description], tablefmt='psql'))

+--------------------+-------------+
|   year(Order_Date) |    sp_month |
|--------------------+-------------|
|               2016 | 6.94679e+06 |
|               2017 | 7.42142e+06 |
|               2018 | 1.2789e+07  |
|               2019 | 1.82644e+07 |
|               2020 | 9.29463e+06 |
|               2021 | 1.03929e+06 |
+--------------------+-------------+


In [24]:
#comparing current_month and previous_month
mycursor.execute("""
select YEAR(Order_Date),month(Order_Date) ,round(sum(pd.Unit_Price_USD*sd.Quantity),2) as sales, LAG(sum(pd.Unit_Price_USD*sd.Quantity))
OVER(order by YEAR(Order_Date), month(Order_Date)) AS Previous_Month_Sales from data_spark.sales_details sd join data_spark.product_details pd 
on sd.ProductKey=pd.ProductKey GROUP BY 
    YEAR(Order_Date), month(Order_Date);
""")
out = mycursor.fetchall()
from tabulate import tabulate
print(tabulate(out, headers=[i[0] for i in mycursor.description], tablefmt='psql'))

+--------------------+---------------------+------------------+------------------------+
|   YEAR(Order_Date) |   month(Order_Date) |            sales |   Previous_Month_Sales |
|--------------------+---------------------+------------------+------------------------|
|               2016 |                   1 | 649919           |                        |
|               2016 |                   2 | 891098           |       649919           |
|               2016 |                   3 | 338407           |       891098           |
|               2016 |                   4 | 110592           |       338407           |
|               2016 |                   5 | 595986           |       110592           |
|               2016 |                   6 | 518585           |       595986           |
|               2016 |                   7 | 454960           |       518585           |
|               2016 |                   8 | 547513           |       454960           |
|               2016 

In [25]:
#comparing current_year and previous_year sales
mycursor.execute("""
select YEAR(Order_Date) as year ,round(sum(pd.Unit_Price_USD*sd.Quantity),2) as sales, LAG(sum(pd.Unit_Price_USD*sd.Quantity))
OVER(order by YEAR(Order_Date)) AS Previous_Year_Sales from data_spark.sales_details sd join data_spark.product_details pd 
on sd.ProductKey=pd.ProductKey GROUP BY YEAR(Order_Date);
""")
out = mycursor.fetchall()
from tabulate import tabulate
print(tabulate(out, headers=[i[0] for i in mycursor.description], tablefmt='psql'))

+--------+-------------+-----------------------+
|   year |       sales |   Previous_Year_Sales |
|--------+-------------+-----------------------|
|   2016 | 6.94679e+06 |                       |
|   2017 | 7.42142e+06 |           6.94679e+06 |
|   2018 | 1.2789e+07  |           7.42142e+06 |
|   2019 | 1.82644e+07 |           1.2789e+07  |
|   2020 | 9.29463e+06 |           1.82644e+07 |
|   2021 | 1.03929e+06 |           9.29463e+06 |
+--------+-------------+-----------------------+


In [26]:
#month wise profit
mycursor.execute("""
select YEAR(Order_Date) as year,month(Order_Date) as month,(SUM(pd.Unit_Price_USD*sd.Quantity) - SUM(pd.Unit_Cost_USD*sd.Quantity)) as sales, 
LAG(sum(pd.Unit_Price_USD*sd.Quantity) - SUM(pd.Unit_Cost_USD*sd.Quantity))
OVER(order by YEAR(Order_Date), month(Order_Date)) AS Previous_Month_Sales,
round(((SUM(pd.Unit_Price_USD*sd.Quantity) - SUM(pd.Unit_Cost_USD*sd.Quantity))-
LAG(sum(pd.Unit_Price_USD*sd.Quantity) - SUM(pd.Unit_Cost_USD*sd.Quantity))
OVER(order by YEAR(Order_Date), month(Order_Date)))/LAG(sum(pd.Unit_Price_USD*sd.Quantity) - SUM(pd.Unit_Cost_USD*sd.Quantity))
OVER(order by YEAR(Order_Date), month(Order_Date))*100,2) as profit_percent
from data_spark.sales_details sd join data_spark.product_details pd 
on sd.ProductKey=pd.ProductKey GROUP BY 
    YEAR(Order_Date), month(Order_Date);
""")
out = mycursor.fetchall()
from tabulate import tabulate
print(tabulate(out, headers=[i[0] for i in mycursor.description], tablefmt='psql'))

+--------+---------+------------------+------------------------+------------------+
|   year |   month |            sales |   Previous_Month_Sales |   profit_percent |
|--------+---------+------------------+------------------------+------------------|
|   2016 |       1 | 388269           |                        |                  |
|   2016 |       2 | 531723           |       388269           |            36.95 |
|   2016 |       3 | 203507           |       531723           |           -61.73 |
|   2016 |       4 |  66231.2         |       203507           |           -67.46 |
|   2016 |       5 | 356049           |        66231.2         |           437.59 |
|   2016 |       6 | 294270           |       356049           |           -17.35 |
|   2016 |       7 | 269369           |       294270           |            -8.46 |
|   2016 |       8 | 325910           |       269369           |            20.99 |
|   2016 |       9 | 331860           |       325910           |            

In [27]:
#year wise profit   
mycursor.execute("""
select YEAR(Order_Date) as Year ,(SUM(pd.Unit_Price_USD*sd.Quantity) - SUM(pd.Unit_Cost_USD*sd.Quantity)) as sales, 
LAG(sum(pd.Unit_Price_USD*sd.Quantity) - SUM(pd.Unit_Cost_USD*sd.Quantity))
OVER(order by YEAR(Order_Date)) AS Previous_year_Sales,
round(((SUM(pd.Unit_Price_USD*sd.Quantity) - SUM(pd.Unit_Cost_USD*sd.Quantity))-
LAG(sum(pd.Unit_Price_USD*sd.Quantity) - SUM(pd.Unit_Cost_USD*sd.Quantity))
OVER(order by YEAR(Order_Date)))/LAG(sum(pd.Unit_Price_USD*sd.Quantity) - SUM(pd.Unit_Cost_USD*sd.Quantity))
OVER(order by YEAR(Order_Date))*100,2) as profit_percent
from data_spark.sales_details sd join data_spark.product_details pd 
on sd.ProductKey=pd.ProductKey GROUP BY 
    YEAR(Order_Date);
""")
out = mycursor.fetchall()
from tabulate import tabulate
print(tabulate(out, headers=[i[0] for i in mycursor.description], tablefmt='psql'))

+--------+------------------+-----------------------+------------------+
|   Year |            sales |   Previous_year_Sales |   profit_percent |
|--------+------------------+-----------------------+------------------|
|   2016 |      4.107e+06   |                       |                  |
|   2017 |      4.33706e+06 |           4.107e+06   |             5.6  |
|   2018 |      7.46496e+06 |           4.33706e+06 |            72.12 |
|   2019 |      1.06977e+07 |           7.46496e+06 |            43.31 |
|   2020 |      5.44746e+06 |           1.06977e+07 |           -49.08 |
|   2021 | 608464           |           5.44746e+06 |           -88.83 |
+--------+------------------+-----------------------+------------------+
